In [1]:
import nibabel as nib
import numpy as np
import os
from os import path
import pandas as pd
from scipy.spatial import distance_matrix
from sklearn.cluster import DBSCAN
import sys
from zipfile import ZipFile

In [2]:
from fsedit_notes_helpers import get_diff_data, summarize_edits, get_bm_edits, get_wm_edits, get_cp_edits, get_bfs_edits

In [3]:
subnum = 'CC0142'
typ = 'wm'
basedir='/Users/zeynepenkavi/Downloads/ConteQC'
outdir='/Users/zeynepenkavi/Downloads/ConteQC'

In [4]:
editp = path.join(basedir, 'sub-'+subnum)
uneditp = path.join(basedir, 'sub-'+subnum+'_unedited')
typ_dict = {'bm': 'brainmask', 'wm': 'wm', 'bfs': 'brain.finalsurfs', 'cp': 'control.dat'}
vol = typ_dict[typ]

In [8]:
get_wm_edits(editp, uneditp, vol, subnum)

,cluster,min_sag,max_sag,min_axe,max_axe,min_cor,max_cor,num_vox,vol,action
0,-1,187,189,144,146,110,114,4,wm,add voxel
1,0,83,93,138,145,54,67,12,wm,add voxel
2,1,151,155,125,129,133,135,7,wm,add voxel
3,2,106,116,122,128,134,143,19,wm,add voxel
0,-1,93,140,124,144,55,141,4,wm,delete voxel
1,0,83,90,135,140,63,67,43,wm,delete voxel
2,1,186,190,144,145,110,114,16,wm,delete voxel
3,2,150,156,126,131,133,136,53,wm,delete voxel
4,3,106,115,121,128,134,142,67,wm,delete voxel


In [7]:
tmp = get_diff_data(editp, uneditp, vol, subnum)
cur_diff_data = tmp.query("Action=='add voxel'")

In [24]:
cur_diff_data.shape

(42, 5)

In [20]:
edit_img = nib.load(path.join(editp,'mri/%s.mgz')%(vol))
edit_data = edit_img.get_fdata()
unedit_img = nib.load(path.join(uneditp,'sub-%s/mri/%s.mgz')%(subnum, vol))
unedit_data = unedit_img.get_fdata()
diff_data = unedit_data - edit_data
    
# Extract non-zero values from difference data and arrange in df
out = pd.DataFrame(np.asarray(np.asarray(diff_data != 0).nonzero()).T).rename(columns={0:"Sag", 1:"Axe", 2:"Cor"})
out['diff_val'] = diff_data[np.where(diff_data != 0)]

In [21]:
out['Action'] = np.where(out.diff_val>0, "delete voxel", "add voxel")

In [26]:
out.query("diff_val != -1")

,Sag,Axe,Cor,diff_val,Action
0,83,136,66,94.0,delete voxel
1,83,137,66,97.0,delete voxel
2,83,137,67,93.0,delete voxel
3,83,138,66,98.0,delete voxel
4,83,138,67,94.0,delete voxel
...,...,...,...,...,...
219,189,145,111,88.0,delete voxel
220,189,145,112,91.0,delete voxel
221,189,145,113,93.0,delete voxel
222,189,145,114,93.0,delete voxel


In [27]:
out

,Sag,Axe,Cor,diff_val,Action
0,83,136,66,94.0,delete voxel
1,83,137,66,97.0,delete voxel
2,83,137,67,93.0,delete voxel
3,83,138,66,98.0,delete voxel
4,83,138,67,94.0,delete voxel
...,...,...,...,...,...
220,189,145,112,91.0,delete voxel
221,189,145,113,93.0,delete voxel
222,189,145,114,93.0,delete voxel
223,189,146,114,-1.0,add voxel


In [ ]:
dm = distance_matrix(cur_diff_data.loc[:,('Sag', 'Axe', 'Cor')], cur_diff_data.loc[:,('Sag', 'Axe', 'Cor')])

In [ ]:
clustering = DBSCAN(eps=50, min_samples=5).fit(dm)

In [ ]:
list(clustering.labels_)

In [ ]:
#cur_diff_data['cluster'] = list(clustering.labels_)
cur_diff_data.insert(0, "cluster", list(clustering.labels_), True)

In [ ]:
cur_diff_data

In [ ]:
summarize_edits(tmp)

In [ ]:
get_bm_edits(editp, uneditp, vol, subnum)

In [ ]:
typ = "cp"
vol = typ_dict[typ]
get_cp_edits(editp, uneditp, vol, subnum)

In [ ]:
typ = "bfs"
vol = typ_dict[typ]
get_bfs_edits(editp, uneditp, vol, subnum)

In [ ]:
fn_dict = {'bm': get_bm_edits, 'wm': get_wm_edits, 'cp': get_cp_edits, 'bfs': get_bfs_edits}
typ = "all"
if typ == 'all':
    out = pd.DataFrame()
    # loop through function lookup dictionary and run all edit extraction functions
    for k,v in fn_dict.items():
        vol = typ_dict[k]
        out = out.append(v(editp=editp, uneditp=uneditp, vol=vol, subnum=subnum))

In [ ]:
pd.set_option('display.max_rows', None)

In [ ]:
out

In [ ]:
import shutil

In [ ]:
from os import path

In [ ]:
subnum = 'CC0076_core1'
basedir='/Users/zeynepenkavi/Downloads/ConteQC'
uneditp = path.join(basedir, 'sub-'+subnum+'_unedited')
shutil.rmtree(uneditp)

To do still: 

next step i need to do: 
refine previous editsm  

can this be useful for that?  
could at least run all subjects through this and see if i've missed writing down any edits i've made  

how should i go about refining edits?
go over mike's reports and compare for each subject how different i am from dorit/mike
run their edited images (taken from box) through 

so timeline:  
run all subjects through this and see if you've missed recording any edits
move onto refinement of previous edits:
    -download dorit's edits from box and make difference images for 
    -overlay original, my edits difference image, dorit's difference image in freeview and scroll through
move onto editing new subjects